# 이미지 내부의 문자 인식(MNIST - 손글씨 숫자 데이터)
 - 데이터 수집: 손글씨 숫자 데이터로 MNIST에서 공개하고 있는 데이터를 사용(http://yann.lecun.com/exdb/mnist/)
     - train-images-idx3-ubyte.gz : 학습 전용 이미지 데이터
     - train-labels-idx1-ubyte.gz : 학습 전용 레이블 데이터
     - t10k-images-idx3-ubyte.gz : 테스트 전용 이미지 데이터
     - t10k-labels-idx1-ubyte.gz : 테스트 전용 레이블 데이터

### Gzip 압축을 해제하는 프로그램

In [3]:
import urllib.request as req
import gzip, os, os.path

savepath = "./mnist"
baseurl = "http://yann.lecun.com/exdb/mnist"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"
]

# 다운로드
if not os.path.exists(savepath): os.mkdir(savepath)
for file in files:
    url = baseurl + "/" + file
    loc = savepath + "/" + file
    print("Download:", url)
    if not os.path.exists(loc):
        req.urlretrieve(url, loc)

# GZip 압축 해제
for file in files:
    gz_file = savepath + "/" + file
    raw_file = savepath + "/" +file.replace(".gz", "")
    print("Gzip: ", file)
    with gzip.open(gz_file, "rb") as fp:
        body = fp.read()
        with open(raw_file, "wb") as w:
            w.write(body)

print ("Ok!!")

Download: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Download: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Download: http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Download: http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Gzip:  train-images-idx3-ubyte.gz
Gzip:  train-labels-idx1-ubyte.gz
Gzip:  t10k-images-idx3-ubyte.gz
Gzip:  t10k-labels-idx1-ubyte.gz
Ok!!


## 바이너리 데이터 분석 후 CSV 파일로 변환

In [3]:
import struct

def to_csv(name, maxdata):
    # 레이블 파일과 이미지 파일 열기
    label_file = open("./mnist/" + name + "-labels-idx1-ubyte", "rb")
    img_file = open("./mnist/" + name + "-images-idx3-ubyte", "rb")
    csv_file = open("./mnist/" + name + ".csv" , "w", encoding="utf-8")
    
    # 헤더 정보 읽기 => 바이너리 처리를 위해 struct 모듈 사용
    # 원하는 바이너리 수만큼 읽어 들이고 정수로 변환 => struct.unpack()
    # Q) 왜 mag 값이 서로 다른데 중첩해서 mag를 쓴거지?
    mag, label_count = struct.unpack(">II", label_file.read(8))
    mag, img_count = struct.unpack(">II", img_file.read(8))
    rows, cols = struct.unpack(">II", img_file.read(8))
    pixels = rows * cols
    
    # 이미지 데이터를 읽고 CSV로 저장하기
    for index in range(label_count):
        if index > maxdata: break
        label = struct.unpack("B", label_file.read(1))[0]
        bdata = img_file.read(pixels)
        sdata = list(map(lambda n: str(n), bdata))
        csv_file.write(str(label) + ",")
        csv_file.write(",".join(sdata) + "\r\n")
        
    # 잘 저장됐는지 이미지 파일로 저장해서 테스트하기
    if index < 10:
        s = "P2 28 28 255\n"
        s += " ".join(sdata)
        iname = "./mnist/{0}-{1}-{2}.pgm".format(name, index, label)
        with open(iname, "w", encoding="utf-8") as f:
            f.write(s)
    
    csv_file.close()
    label_file.close()
    img_file.close()

to_csv("train", 9999)
to_csv("t10k", 500)

## 이미지 데이터 학습시키기
- 이미지 데이터를 사용해 머신러닝 => 이미지 데이터를 어떻게 입력해야 좋을지 생각!!

### 손글씨 숫자 데이터를 어떻게 벡터로 변환할 수 있을까?
* 이미지 픽셀 데이터를 그대로 벡터로 넣는 방법

### 이미지 픽셀 데이터 -> 24x24(576)의 벡터로 그대로 넣어 학습시키는 프로그램
1. CSV 파일에서 학습 데이터와 테스트 데이터를 읽는다.
2. 학습 데이터를 사용해 이미지 픽셀을 학습.
3. 테스트 데이터를 활용해 예측
4. 예측 결과와 답을 비교해서 정답률을 구한다.

In [4]:
from sklearn import model_selection, svm, metrics

# CSV 파일을 읽어 들이고 가공하기
def load_csv(filename):
    labels = []
    images = []
    
    with open(filename, "r") as f:
        for line in f:
            cols = line.split(",")
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            # 이미지 데이터의 각 픽셀은 0~255까지의 정수, 이를 256으로 나눔으로써 0 이상 1 미만인 '실수 벡터' 가 된다.
            vals = list(map(lambda n: int(n) / 256, cols)) 
            images.append(vals)
    return {"labels": labels, "images": images}

data = load_csv("./mnist/train.csv")
test = load_csv("./mnist/t10k.csv")

# 학습하기
clf = svm.SVC()
clf.fit(data["images"], data["labels"])

# 예측하기
predict = clf.predict(test["images"])

# 결과 확인하기
ac_score = metrics.accuracy_score(test["labels"], predict)
cl_report = metrics.classification_report(test["labels"], predict)
print("정답률 = ", ac_score)
print("리포트 = ")
print(cl_report)


정답률 =  0.9560878243512974
리포트 = 
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        42
           1       1.00      1.00      1.00        67
           2       0.98      0.96      0.97        55
           3       0.93      0.91      0.92        46
           4       0.93      0.98      0.96        55
           5       0.92      0.94      0.93        50
           6       0.98      0.93      0.95        43
           7       0.94      0.96      0.95        49
           8       0.93      0.97      0.95        40
           9       1.00      0.91      0.95        54

    accuracy                           0.96       501
   macro avg       0.95      0.95      0.95       501
weighted avg       0.96      0.96      0.96       501

